# Preprocess

### Install necessary packages

- sentence-transformers            2.2.2
- elasticsearch                    8.10.0

In [ ]:
# !pip install sentence_transformers
# !pip install elasticsearch

### Import necessary packages

In [ ]:
import torch
import pandas as pd
import numpy as np
import json
import os
import uuid

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers

from tqdm.auto import tqdm
tqdm.pandas()

### Test Ransaka/sinhala-roberta-sentence-transformer model

In [ ]:
sentences = ["වර්ජින් ඕස්ට්‍රේලියාව ක්‍රියාත්මක වීමට පටන් ගත්තේ කවදාද?"]

model = SentenceTransformer('Ransaka/sinhala-roberta-sentence-transformer')
embeddings = model.encode(sentences)
print(embeddings)

[[-1.88468337e+00 -9.25925672e-01  4.96275753e-01  3.41932982e-01
  -1.16093600e+00 -4.34460759e-01  2.28524184e+00 -1.44768071e+00
   6.86134100e-01  2.33392501e+00 -9.45740402e-01 -7.54496038e-01
  -8.59015107e-01  1.13579726e+00 -1.53073454e+00 -3.50521982e-01
   9.48073924e-01 -1.83459902e+00  6.18936598e-01 -4.10729289e-01
   1.35741115e+00  2.46794641e-01 -7.39638321e-03 -4.84763891e-01
  -2.93065578e-01  1.56439736e-01 -1.04954541e+00  3.65333408e-01
   8.80976200e-01 -1.64959836e+00 -1.18835318e+00  9.09197390e-01
   8.85036826e-01 -1.84392488e+00  8.40911642e-02  1.70922592e-01
   8.95147860e-01 -7.54095912e-02  8.94138098e-01  1.17550898e+00
  -1.08640671e+00  2.20424250e-01 -3.03809298e-03  2.60784054e+00
  -8.72547507e-01  1.14982319e+00 -1.21674478e+00  1.82807672e+00
  -1.04311419e+00  1.29513562e-01 -6.91705823e-01 -1.95930719e+00
  -1.49430007e-01 -2.97946602e-01 -3.87951493e-01 -6.73543870e-01
  -1.17944038e+00 -3.85963321e-01 -1.35346663e+00  4.14878458e-01
  -4.08074

In [ ]:
embeddings.shape

(1, 768)

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

data = {
    "question": [
        "ප්‍රංශයේ අගනුවර කුමක්ද?",
        "රෝමියෝ ජුලියට් ලිව්වේ කවුද?",
        "ප්‍රභාසංශ්ලේෂණය ක්‍රියා කරන්නේ කෙසේද?",
        "Python හි ප්‍රධාන ලක්ෂණ මොනවාද?",
        "අපේ සෞරග්‍රහ මණ්ඩලයේ විශාලතම ග්‍රහලෝකය කුමක්ද?",
        "මම චොකලට් කේක් එකක් පුළුස්සන්නේ කෙසේද?",
        "රවුමක වර්ගඵලය ගණනය කිරීමේ සූත්‍රය කුමක්ද?",
        "පරිගණක CPU වැඩ කරන්නේ කෙසේද?",
        "COVID-19 රෝග ලක්ෂණ මොනවාද?",
        "පෘථිවිය සහ සඳ අතර ඇති දුර කුමක්ද?"
    ],
    "answers": [
        "ප්රංශයේ අගනුවර පැරිස් වේ.",
        "විලියම් ෂේක්ස්පියර් 'රෝමියෝ ජුලියට්' ලිව්වා.",
        "ප්‍රභාසංශ්ලේෂණය යනු ශාක සූර්යාලෝකය ශක්තිය බවට පරිවර්තනය කරන ක්‍රියාවලියයි.",
        "Python එහි සරල බව සහ කියවීමේ හැකියාව සඳහා ප්‍රසිද්ධය.",
        "බ්‍රහස්පති අපේ සෞරග්‍රහ මණ්ඩලයේ විශාලතම ග්‍රහලෝකයයි.",
        "චොකලට් කේක් එකක් පිළිස්සීම සඳහා, ඔබට කොකෝවා, සීනි සහ පිටි වැනි අමුද්රව්ය අවශ්ය වනු ඇත.",
        "රවුමක වර්ගඵලය ගණනය කිරීමේ සූත්‍රය A = πr^2 වේ.",
        "පරිගණක CPU එකක් උපදෙස් ක්‍රියාවට නංවා ගණනය කිරීම් සිදු කරයි.",
        "COVID-19 හි පොදු රෝග ලක්ෂණ උණ, කැස්ස සහ හුස්ම හිරවීම ඇතුළත් වේ.",
        "පෘථිවිය සහ සඳ අතර සාමාන්‍ය දුර සැතපුම් 238,855 (කිලෝමීටර් 384,400) පමණ වේ."
    ]
}

df = pd.DataFrame(data)

new_question = "ප්‍රංශයේ නගරයේ ජනගහනය කොපමණද?"
embeddings = model.encode(df['question'].tolist() + [new_question])
cosine_similarities = cosine_similarity([embeddings[-1]], embeddings[:-1])[0]
df['cosine_similarity'] = cosine_similarities
df.sort_values(by='cosine_similarity', ascending=False)

,question,answers,cosine_similarity
4,අපේ සෞරග්‍රහ මණ්ඩලයේ විශාලතම ග්‍රහලෝකය කුමක්ද?,බ්‍රහස්පති අපේ සෞරග්‍රහ මණ්ඩලයේ විශාලතම ග්‍රහල...,0.758709
3,Python හි ප්‍රධාන ලක්ෂණ මොනවාද?,Python එහි සරල බව සහ කියවීමේ හැකියාව සඳහා ප්‍ර...,0.700786
9,පෘථිවිය සහ සඳ අතර ඇති දුර කුමක්ද?,"පෘථිවිය සහ සඳ අතර සාමාන්‍ය දුර සැතපුම් 238,855...",0.681984
6,රවුමක වර්ගඵලය ගණනය කිරීමේ සූත්‍රය කුමක්ද?,රවුමක වර්ගඵලය ගණනය කිරීමේ සූත්‍රය A = πr^2 වේ.,0.635862
0,ප්‍රංශයේ අගනුවර කුමක්ද?,ප්රංශයේ අගනුවර පැරිස් වේ.,0.576573
8,COVID-19 රෝග ලක්ෂණ මොනවාද?,"COVID-19 හි පොදු රෝග ලක්ෂණ උණ, කැස්ස සහ හුස්ම ...",0.573074
2,ප්‍රභාසංශ්ලේෂණය ක්‍රියා කරන්නේ කෙසේද?,ප්‍රභාසංශ්ලේෂණය යනු ශාක සූර්යාලෝකය ශක්තිය බවට ...,0.570587
5,මම චොකලට් කේක් එකක් පුළුස්සන්නේ කෙසේද?,"චොකලට් කේක් එකක් පිළිස්සීම සඳහා, ඔබට කොකෝවා, ස...",0.533898
1,රෝමියෝ ජුලියට් ලිව්වේ කවුද?,විලියම් ෂේක්ස්පියර් 'රෝමියෝ ජුලියට්' ලිව්වා.,0.533483
7,පරිගණක CPU වැඩ කරන්නේ කෙසේද?,පරිගණක CPU එකක් උපදෙස් ක්‍රියාවට නංවා ගණනය කිර...,0.467700


### Import dataset

In [ ]:
poem_csv_path = '/content/poem_corpus.csv'
poem_df = pd.read_csv(poem_csv_path)

In [ ]:
poem_df.columns = poem_df.columns.str.strip().str.lower()

In [ ]:
poem_df.head()

,poem name,poet,poem number,line,metaphor present or not,count of the metaphor,metaphorical terms sinhala,metaphorical terms english,metaphorical meaning sinhala,metaphorical meaning english
0,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සෙත් සිරි දෙන මහ ගුණ මුහුදාණන්,yes,1,මහ ගුණ මුහුදාණන්,great ocean of noble qualities,මහා සාගරය මෙන් ඉමක් කොනක් නැති උතුම් ගුණ,Noble qualities that are as endless as the gre...
1,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සත් හට වන බව දුකට වෙදාණන්,yes,1,දුකට වෙදාණන්,physician for sadness,ලෝක සත්ත්වයාට ඇතිවන ලෙඩ දුක් මරණය ආදී සසර දුකට...,He is like a great physician for the world's s...
2,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,තිත් ගණදුරු දුරලන දිනිදාණන්,yes,1,තිත් ගණදුරු දුරලන දිනිදාණන්,Sun who removes evil spirits,මිසදිටු නම් මහ අඳුර දුර කිරීමට සූර්යයා වැනි බු...,Lord Buddha is like the sun to dispel the grea...
3,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සිත් සතොසින් නමදිමි මුනිදාණන්,no,0,NaN,NaN,NaN,NaN
4,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,15,බලකොටු වන් තර මෙසසර නුවරේ,yes,1,බලකොටු වන් තර මෙසසර නුවරේ,city of repeated living and dying,සංසාරය නැමති නගරයේ හි ස්ථිර බලකොටුවක් වැනි,Like a permanent fortress in the city of 'Sams...


In [ ]:
# Restructure the format
poem_df['poem'] = poem_df.groupby(['poem name',	'poet', 'poem number'])['line'].transform(lambda poem_line: '\n'.join(poem_line))
poem_df['poem'] = poem_df.apply(lambda row: row['poem'] if row['count of the metaphor'] > 0 else row['line'], axis=1)
poem_df['metaphorical terms english'] = poem_df['metaphorical terms english'].str.lower()
poem_df['metaphorical meaning english'] = poem_df['metaphorical meaning english'].str.lower()

# Extract required columns
poem_set = poem_df[[
    "poem name",
    "poet",
    "poem number",
    "poem",
    "count of the metaphor",
    "metaphorical terms sinhala",
    "metaphorical terms english",
    "metaphorical meaning sinhala",
    "metaphorical meaning english"
    ]]

In [ ]:
poem_set['count of the metaphor'] = poem_set['count of the metaphor'].astype(int)
poem_set['poem number'] = poem_set['poem number'].astype(int)

<ipython-input-17-2a8ae7f11a33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poem_set['count of the metaphor'] = poem_set['count of the metaphor'].astype(int)
<ipython-input-17-2a8ae7f11a33>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poem_set['poem number'] = poem_set['poem number'].astype(int)


In [ ]:
poem_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   poem name                     236 non-null    object
 1   poet                          236 non-null    object
 2   poem number                   236 non-null    int64 
 3   poem                          236 non-null    object
 4   count of the metaphor         236 non-null    int64 
 5   metaphorical terms sinhala    98 non-null     object
 6   metaphorical terms english    98 non-null     object
 7   metaphorical meaning sinhala  98 non-null     object
 8   metaphorical meaning english  98 non-null     object
dtypes: int64(2), object(7)
memory usage: 16.7+ KB


In [ ]:
# Drop non metaphor poem lines
metaphor_poem_df = poem_set[poem_set['count of the metaphor'] > 0]

In [ ]:
metaphor_poem_df['formatted_info'] = "කවියේ නම:\n" + metaphor_poem_df['poem name'] + "\n\nකවියා:\n" + metaphor_poem_df['poet'] + "\n\nකවිය:\n" + metaphor_poem_df['poem'] + "\n\nරූපක පදය:\n" + metaphor_poem_df['metaphorical terms sinhala']

<ipython-input-20-d82a5bce59d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metaphor_poem_df['formatted_info'] = "කවියේ නම:\n" + metaphor_poem_df['poem name'] + "\n\nකවියා:\n" + metaphor_poem_df['poet'] + "\n\nකවිය:\n" + metaphor_poem_df['poem'] + "\n\nරූපක පදය:\n" + metaphor_poem_df['metaphorical terms sinhala']


In [ ]:
metaphor_poem_df.head()

,poem name,poet,poem number,poem,count of the metaphor,metaphorical terms sinhala,metaphorical terms english,metaphorical meaning sinhala,metaphorical meaning english,formatted_info
0,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සෙත් සිරි දෙන මහ ගුණ මුහුදාණන්\nසත් හට වන බව ද...,1,මහ ගුණ මුහුදාණන්,great ocean of noble qualities,මහා සාගරය මෙන් ඉමක් කොනක් නැති උතුම් ගුණ,noble qualities that are as endless as the gre...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...
1,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සෙත් සිරි දෙන මහ ගුණ මුහුදාණන්\nසත් හට වන බව ද...,1,දුකට වෙදාණන්,physician for sadness,ලෝක සත්ත්වයාට ඇතිවන ලෙඩ දුක් මරණය ආදී සසර දුකට...,he is like a great physician for the world's s...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...
2,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සෙත් සිරි දෙන මහ ගුණ මුහුදාණන්\nසත් හට වන බව ද...,1,තිත් ගණදුරු දුරලන දිනිදාණන්,sun who removes evil spirits,මිසදිටු නම් මහ අඳුර දුර කිරීමට සූර්යයා වැනි බු...,lord buddha is like the sun to dispel the grea...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...
4,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,15,බලකොටු වන් තර මෙසසර නුවරේ\nමුල් සැටු වන් දළු ල...,1,බලකොටු වන් තර මෙසසර නුවරේ,city of repeated living and dying,සංසාරය නැමති නගරයේ හි ස්ථිර බලකොටුවක් වැනි,like a permanent fortress in the city of 'sams...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...
5,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,15,බලකොටු වන් තර මෙසසර නුවරේ\nමුල් සැටු වන් දළු ල...,1,මුල් සැටු වන් දළු ලන පව් පඳුරේ,sprouts again and again on the mat of sin,පාප නමැති පැදුරේ යළි යළිත් දළුලන ඇසටු මුලක් මෙන්,like a root that sprouts again and again on th...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...


In [ ]:
metaphor_count = poem_set['count of the metaphor'].sum()
print(f"metaphor count: {metaphor_count}")

metaphor count: 100


### Get embeddings

In [ ]:
class Tokenizer(object):
    def __init__(self):
        self.model = SentenceTransformer('Ransaka/sinhala-roberta-sentence-transformer')

    def get_token(self, documents):
        sentences  = [documents]
        sentence_embeddings = self.model.encode(sentences)
        encod_np_array = np.array(sentence_embeddings)
        encod_list = encod_np_array.tolist()
        return encod_list[0]

In [ ]:
token_instance = Tokenizer()

In [ ]:
df = metaphor_poem_df.copy()

In [ ]:
df['embeddings'] = df['formatted_info'].progress_apply(token_instance.get_token)

  0%|          | 0/98 [00:00<?, ?it/s]

In [ ]:
df.head()

,poem name,poet,poem number,poem,count of the metaphor,metaphorical terms sinhala,metaphorical terms english,metaphorical meaning sinhala,metaphorical meaning english,formatted_info,embeddings
0,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සෙත් සිරි දෙන මහ ගුණ මුහුදාණන්\nසත් හට වන බව ද...,1,මහ ගුණ මුහුදාණන්,great ocean of noble qualities,මහා සාගරය මෙන් ඉමක් කොනක් නැති උතුම් ගුණ,noble qualities that are as endless as the gre...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...,"[-0.36453789472579956, -1.870797038078308, 1.5..."
1,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සෙත් සිරි දෙන මහ ගුණ මුහුදාණන්\nසත් හට වන බව ද...,1,දුකට වෙදාණන්,physician for sadness,ලෝක සත්ත්වයාට ඇතිවන ලෙඩ දුක් මරණය ආදී සසර දුකට...,he is like a great physician for the world's s...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...,"[-0.34118983149528503, -1.851097822189331, 1.5..."
2,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සෙත් සිරි දෙන මහ ගුණ මුහුදාණන්\nසත් හට වන බව ද...,1,තිත් ගණදුරු දුරලන දිනිදාණන්,sun who removes evil spirits,මිසදිටු නම් මහ අඳුර දුර කිරීමට සූර්යයා වැනි බු...,lord buddha is like the sun to dispel the grea...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...,"[-0.36715346574783325, -1.868776798248291, 1.5..."
4,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,15,බලකොටු වන් තර මෙසසර නුවරේ\nමුල් සැටු වන් දළු ල...,1,බලකොටු වන් තර මෙසසර නුවරේ,city of repeated living and dying,සංසාරය නැමති නගරයේ හි ස්ථිර බලකොටුවක් වැනි,like a permanent fortress in the city of 'sams...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...,"[-1.0431283712387085, -2.1417312622070312, 1.4..."
5,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,15,බලකොටු වන් තර මෙසසර නුවරේ\nමුල් සැටු වන් දළු ල...,1,මුල් සැටු වන් දළු ලන පව් පඳුරේ,sprouts again and again on the mat of sin,පාප නමැති පැදුරේ යළි යළිත් දළුලන ඇසටු මුලක් මෙන්,like a root that sprouts again and again on th...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...,"[-0.9584224224090576, -2.1635382175445557, 1.3..."


In [ ]:
memory_usage_bytes = df.memory_usage(deep=True).sum()
memory_usage_mb = memory_usage_bytes / (1024**2)

print(f"Memory Usage: {memory_usage_mb} MB")

Memory Usage: 0.7768793106079102 MB


In [ ]:
embedding_dimension = model.get_sentence_embedding_dimension()
embedding_dimension

768

### Get similarity score

In [ ]:
new_question = ["මහ ගුණ මුහුදාණන්රූපක පදයේ තේරුම කුමක්ද?"]
new_question_embedding = model.encode(new_question)
df['cosine_similarity'] = df['embeddings'].apply(
    lambda x: cosine_similarity(np.array([x]), new_question_embedding)[0][0]
)
df = df.sort_values(by='cosine_similarity', ascending=False)
df = df.reset_index(drop=True)
df[['cosine_similarity', 'poem', 'metaphorical terms sinhala']]

,cosine_similarity,poem,metaphorical terms sinhala
0,0.879438,නිදහස මහ මුහුදක් වේ\nඑහි උල්පත පුත නුඹ වේ\nඒ බ...,මහ මුහුදක් වේ
1,0.873124,ගඟ යන ඇත් කුණු ඉඳ මස් රසටා\nසිඳු මැදැ වැදැ වැන...,ගඟ යන ඇත් කුණු ඉඳ මස් රසටා සිඳු මැදැ වැදැ වැනස...
2,0.868003,අවිස නොවන බස් කණට යවුල් ලෙස\nසිතස තොසින් කිය ක...,කණට යවුල් ලෙස
3,0.865742,තැතකින් ගෙන දැමුවත් කැට නුඹ කුස\nඅතරක නොරඳන මෙ...,තැතකින් ගෙන දැමුවත් කැට නුඹ කුස
4,0.865058,නිදහස මහ මුහුදක් වේ\nඑහි උල්පත පුත නුඹ වේ\nඒ බ...,එහි උල්පත
...,...,...,...
61,0.745051,බව රෑ කෙලෙස් සඳ \nකිරණින් මුහුලු බඹ සඳ\nපියු...,"බව රෑ, කෙලෙස් සඳ"
62,0.743233,නට නොත මතඟනක පිළිබිඹුනි එ පුරේහි\nපිළිමිණිගෙ ප...,නට නොත මතඟනක පිළිබිඹුනි
63,0.739987,දසවමියන් විසිතුරු රුදු පියයුරු මඬලේ \nදුලු පළ ...,දුලු පළ කොකුමඟරා ගී වී තෙදෙ නරවරා
64,0.737824,මැඩියන් නයි මුව ගොදුරු වැ ඉදයා\nපණුවන් කන මෙන්...,මැඩියන් නයි මුව ගොදුරු වැ ඉදයා පණුවන් කන මෙන්


In [ ]:
new_question = ["මහ ගුණ මුහුදාණන්රූපක පදයේ තේරුම කුමක්ද?"]
new_question_embedding = model.encode(new_question)
df['dot_product_similarity'] = df['embeddings'].apply(lambda x: np.dot(x, new_question_embedding[0]))

df = df.sort_values(by='dot_product_similarity', ascending=False)
df = df.reset_index(drop=True)
df[['dot_product_similarity', 'poem', 'metaphorical terms sinhala']]

,dot_product_similarity,poem,metaphorical terms sinhala
0,746.082818,නිදහස මහ මුහුදක් වේ\nඑහි උල්පත පුත නුඹ වේ\nඒ බ...,මහ මුහුදක් වේ
1,742.099018,තැතකින් ගෙන දැමුවත් කැට නුඹ කුස\nඅතරක නොරඳන මෙ...,තැතකින් ගෙන දැමුවත් කැට නුඹ කුස
2,740.486549,අවිස නොවන බස් කණට යවුල් ලෙස\nසිතස තොසින් කිය ක...,කණට යවුල් ලෙස
3,738.763057,ගඟ යන ඇත් කුණු ඉඳ මස් රසටා\nසිඳු මැදැ වැදැ වැන...,ගඟ යන ඇත් කුණු ඉඳ මස් රසටා සිඳු මැදැ වැදැ වැනස...
4,733.621066,නිදහස මහ මුහුදක් වේ\nඑහි උල්පත පුත නුඹ වේ\nඒ බ...,එහි උල්පත
...,...,...,...
61,634.796666,නට නොත මතඟනක පිළිබිඹුනි එ පුරේහි\nපිළිමිණිගෙ ප...,නට නොත මතඟනක පිළිබිඹුනි
62,634.748418,බව රෑ කෙලෙස් සඳ \nකිරණින් මුහුලු බඹ සඳ\nපියු...,"බව රෑ, කෙලෙස් සඳ"
63,631.177833,දසවමියන් විසිතුරු රුදු පියයුරු මඬලේ \nදුලු පළ ...,දුලු පළ කොකුමඟරා ගී වී තෙදෙ නරවරා
64,622.514145,මැඩියන් නයි මුව ගොදුරු වැ ඉදයා\nපණුවන් කන මෙන්...,මැඩියන් නයි මුව ගොදුරු වැ ඉදයා පණුවන් කන මෙන්


In [ ]:
new_question = ["මහ ගුණ මුහුදාණන්රූපක පදයේ තේරුම කුමක්ද?"]
new_question_embedding = model.encode(new_question)
df['l2_norm_similarity'] = df['embeddings'].apply(lambda x: np.linalg.norm(x - new_question_embedding[0]))

df = df.sort_values(by='l2_norm_similarity', ascending=True)
df = df.reset_index(drop=True)
df[['l2_norm_similarity', 'poem', 'metaphorical terms sinhala']]

,l2_norm_similarity,poem,metaphorical terms sinhala
0,14.335165,නිදහස මහ මුහුදක් වේ\nඑහි උල්පත පුත නුඹ වේ\nඒ බ...,මහ මුහුදක් වේ
1,14.689897,ගඟ යන ඇත් කුණු ඉඳ මස් රසටා\nසිඳු මැදැ වැදැ වැන...,ගඟ යන ඇත් කුණු ඉඳ මස් රසටා සිඳු මැදැ වැදැ වැනස...
2,15.028791,අවිස නොවන බස් කණට යවුල් ලෙස\nසිතස තොසින් කිය ක...,කණට යවුල් ලෙස
3,15.160263,නිදහස මහ මුහුදක් වේ\nඑහි උල්පත පුත නුඹ වේ\nඒ බ...,එහි උල්පත
4,15.186023,තැතකින් ගෙන දැමුවත් කැට නුඹ කුස\nඅතරක නොරඳන මෙ...,තැතකින් ගෙන දැමුවත් කැට නුඹ කුස
...,...,...,...
61,20.859663,බව රෑ කෙලෙස් සඳ \nකිරණින් මුහුලු බඹ සඳ\nපියු...,"බව රෑ, කෙලෙස් සඳ"
62,20.957325,නට නොත මතඟනක පිළිබිඹුනි එ පුරේහි\nපිළිමිණිගෙ ප...,නට නොත මතඟනක පිළිබිඹුනි
63,21.063461,මැඩියන් නයි මුව ගොදුරු වැ ඉදයා\nපණුවන් කන මෙන්...,මැඩියන් නයි මුව ගොදුරු වැ ඉදයා පණුවන් කන මෙන්
64,21.076557,දසවමියන් විසිතුරු රුදු පියයුරු මඬලේ \nදුලු පළ ...,දුලු පළ කොකුමඟරා ගී වී තෙදෙ නරවරා


### Connect to local elasticsearch

In [ ]:
ENDPOINT = "https://fb83-2401-dd00-10-20-24fe-78c5-d7ed-6f7d.ngrok.io:443"
USERNAME = "elastic"
PASSWORD = "WRpT827K5LnRWFBUZh6r"

es = Elasticsearch(hosts=[ENDPOINT],  http_auth=(USERNAME, PASSWORD), timeout=300)
es.ping()

<ipython-input-34-318fac2b795a>:5: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(hosts=[ENDPOINT],  http_auth=(USERNAME, PASSWORD), timeout=300)
<ipython-input-34-318fac2b795a>:5: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(hosts=[ENDPOINT],  http_auth=(USERNAME, PASSWORD), timeout=300)


True

### Create an index

In [ ]:
df.head()

,poem name,poet,poem number,poem,count of the metaphor,metaphorical terms sinhala,metaphorical terms english,metaphorical meaning sinhala,metaphorical meaning english,formatted_info,embeddings
0,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සෙත් සිරි දෙන මහ ගුණ මුහුදාණන්\nසත් හට වන බව ද...,1,මහ ගුණ මුහුදාණන්,great ocean of noble qualities,මහා සාගරය මෙන් ඉමක් කොනක් නැති උතුම් ගුණ,noble qualities that are as endless as the gre...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...,"[-0.36453789472579956, -1.870797038078308, 1.5..."
1,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සෙත් සිරි දෙන මහ ගුණ මුහුදාණන්\nසත් හට වන බව ද...,1,දුකට වෙදාණන්,physician for sadness,ලෝක සත්ත්වයාට ඇතිවන ලෙඩ දුක් මරණය ආදී සසර දුකට...,he is like a great physician for the world's s...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...,"[-0.34118983149528503, -1.851097822189331, 1.5..."
2,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සෙත් සිරි දෙන මහ ගුණ මුහුදාණන්\nසත් හට වන බව ද...,1,තිත් ගණදුරු දුරලන දිනිදාණන්,sun who removes evil spirits,මිසදිටු නම් මහ අඳුර දුර කිරීමට සූර්යයා වැනි බු...,lord buddha is like the sun to dispel the grea...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...,"[-0.36715346574783325, -1.868776798248291, 1.5..."
4,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,15,බලකොටු වන් තර මෙසසර නුවරේ\nමුල් සැටු වන් දළු ල...,1,බලකොටු වන් තර මෙසසර නුවරේ,city of repeated living and dying,සංසාරය නැමති නගරයේ හි ස්ථිර බලකොටුවක් වැනි,like a permanent fortress in the city of 'sams...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...,"[-1.0431283712387085, -2.1417312622070312, 1.4..."
5,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,15,බලකොටු වන් තර මෙසසර නුවරේ\nමුල් සැටු වන් දළු ල...,1,මුල් සැටු වන් දළු ලන පව් පඳුරේ,sprouts again and again on the mat of sin,පාප නමැති පැදුරේ යළි යළිත් දළුලන ඇසටු මුලක් මෙන්,like a root that sprouts again and again on th...,කවියේ නම:\nලෝවැඩ සඟරාව\n\nකවියා:\nවීදාගම මෙත් ...,"[-0.9584224224090576, -2.1635382175445557, 1.3..."


@TODO add year as well

In [ ]:
# PUT posting
config = {
   "settings":{
      "number_of_shards":1,
      "number_of_replicas":0
   },
   "mappings":{
      "properties":{
         "embeddings":{
            "type": "dense_vector",
            "dims": 768,
            "index": True,
            "similarity": "cosine"
         },
         "poem_name":{
            "type":"text"
         },
         "poet":{
            "type":"text",
         },
         "poem_number":{
            "type":"text"
         },
         "poem":{
            "type":"text"
         },
         "metaphorical_terms_si":{
            "type":"text"
         },
         "metaphorical_terms_en":{
            "type":"text"
         },
         "metaphorical_meaning_si":{
            "type":"text"
         },
         "metaphorical_meaning_en":{
            "type":"text"
         },
      }
   }
}

In [ ]:
es.indices.create(
    index="poem",
    settings=config["settings"],
    mappings=config["mappings"],
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'poem'})

In [ ]:
print(es.indices.exists(index=["poem"]))

True


### Insert data to local elasticsearch

In [ ]:
df[df.isna().any(axis=1)]

,poem name,poet,poem number,poem,count of the metaphor,metaphorical terms sinhala,metaphorical terms english,metaphorical meaning sinhala,metaphorical meaning english,formatted_info,embeddings


In [ ]:
df = df.dropna()

In [ ]:
for i,row in df.iterrows():
    try:
      doc = {
          "poem_name": row["poem name"],
          "poet": row["poet"],
          "poem_number": row["poem number"],
          "poem": row["poem"],
          "metaphorical_terms_si": row["metaphorical terms sinhala"],
          "metaphorical_terms_en": row["metaphorical terms english"],
          "metaphorical_meaning_si": row["metaphorical meaning sinhala"],
          "metaphorical_meaning_en": row["metaphorical meaning english"],
          "embeddings": row["embeddings"]
      }
      es.index(index="poem", id=i, document=doc)
    except Exception as e:
      print(print(f"An error occurred: {e}"))
      pass

### Bulk experimental

In [ ]:
bulk_data = []

try:
  for i,row in df.iterrows():
      bulk_data.append(
          {
              "_index": "poem",
              "_id": i,
              "_source": {
                  "poem_name": row["poem name"],
                  "poem_number": row["poem number"],
                  "poem": row["poem"],
                  "metaphorical_terms_si": row["metaphorical terms sinhala"],
                  "metaphorical_terms_en": row["metaphorical terms english"],
                  "metaphorical_meaning_si": row["metaphorical meaning sinhala"],
                  "metaphorical_meaning_english": row["metaphorical meaning english"],
                  "embeddings": row["embeddings"]
              }
          }
      )

  helpers.bulk(es, bulk_data)
except helpers.BulkIndexError as e:
    for error in e.errors:
        print("Error:", error)

[document on bulk insert](https://dylancastillo.co/elasticsearch-python/#:~:text=You%20can%20use%20es.,items%20at%20the%20same%20time.)

### Query elasticsearch

In [ ]:
query  = ["මහ ගුණ මුහුදාණන්රූපක පදයේ තේරුම කුමක්ද?"]
sentence_embeddings = model.encode(query)
encod_np_array = np.array(sentence_embeddings)
encod_list = encod_np_array.tolist()
query_embeddings = encod_list[0]

In [ ]:
query_embeddings

In [ ]:
knn_search_params = {
    "knn": {
        "field": "embeddings",
        "query_vector": query_embeddings,
        "k": 50,
        "num_candidates": 1000
    },
    "_source": [
        "poem_name",
        "poet",
        "poem_number",
        "poem",
        "metaphorical_terms_si",
        "metaphorical_terms_en",
        "metaphorical_meaning_si",
        "metaphorical_meaning_en"
        ]
}

# Perform the KNN search
results = es.knn_search(index="poem", body=knn_search_params)

print(results)

<ipython-input-58-9d1e1d222611>:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.knn_search(index="poem", body=knn_search_params)


{'took': 29, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 65, 'relation': 'eq'}, 'max_score': 0.93971914, 'hits': [{'_index': 'poem', '_id': '0', '_score': 0.93971914, '_source': {'poem_name': 'ලෝවැඩ සඟරාව', 'poet': 'වීදාගම මෙත් හිමිපාණන්', 'poem_number': 2, 'poem': 'නිදහස මහ මුහුදක් වේ\nඑහි උල්පත පුත නුඹ වේ\nඒ බව සිහිකොට මෙලොවේ\nයුතුකම ඉටු කළ යුතු වේ', 'metaphorical_terms_si': 'මහ මුහුදක් වේ', 'metaphorical_terms_en': 'a big sea', 'metaphorical_meaning_si': 'නිදහස නැමැති සාගරයයි', 'metaphorical_meaning_en': 'Freedom is a great sea'}}, {'_index': 'poem', '_id': '1', '_score': 0.9365618, '_source': {'poem_name': 'ලෝවැඩ සඟරාව', 'poet': 'වීදාගම මෙත් හිමිපාණන්', 'poem_number': 64, 'poem': 'ගඟ යන ඇත් කුණු ඉඳ මස් රසටා\nසිඳු මැදැ වැදැ වැනසුණු මෙන් කපුටා\nලද ඉසුරෙහි ලොබ කළහොත් තොපටා\nබව සයුරෙන් ගමනෙක් නැත ගොඩටා', 'metaphorical_terms_si': 'ගඟ යන ඇත් කුණු ඉඳ මස් රසටා සිඳු මැදැ වැදැ වැනසුණු මෙන් කපුටා', 'metaphorical_terms_en

<ipython-input-58-9d1e1d222611>:21: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  results = es.knn_search(index="poem", body=knn_search_params)


In [ ]:
similar_results = [x["_source"]  for x in results['hits']['hits']]
len(similar_results)

50

In [ ]:
data = []
for hit in results['hits']['hits']:
    _score = hit['_score']
    poem_name = hit['_source']["poem_name"]
    poet = hit['_source']["poet"]
    poem_number = hit['_source']["poem_number"]
    poem = hit['_source']["poem"]
    metaphorical_terms_si = hit['_source']["metaphorical_terms_si"]
    metaphorical_terms_en = hit['_source']["metaphorical_terms_en"]
    metaphorical_meaning_si = hit['_source']["metaphorical_meaning_si"]
    metaphorical_meaning_en = hit['_source']["metaphorical_meaning_en"]

    entry = {
        "_score": _score,
        "poem name": poem_name,
        "poet": poet,
        "poem number": poem_number,
        "poem": poem,
        "metaphorical terms sinhala": metaphorical_terms_si,
        "metaphorical terms english": metaphorical_terms_en,
        "metaphorical meaning sinhala": metaphorical_meaning_si,
        "metaphorical meaning english": metaphorical_meaning_en
    }

    data.append(entry)

# Create a pandas DataFrame
dfr = pd.DataFrame(data)
dfr.sort_values(by='_score', ascending=False)
dfr

### Ranking

In [ ]:
dfr_ranking = dfr.copy()

In [ ]:
data_embeddings_dfr = model.encode(dfr_ranking['metaphorical terms sinhala'].tolist()+ dfr_ranking['poet'].tolist() + dfr_ranking['poem'].tolist() + dfr_ranking['poem name'].tolist())
# data_embeddings_dfr = model.encode(dfr_ranking['metaphorical terms sinhala'].tolist() + dfr_ranking['poem'].tolist() + dfr_ranking['poem name'].tolist())

In [ ]:
similarity_scores_terms = util.pytorch_cos_sim(query_embeddings, model.encode(dfr_ranking['metaphorical terms sinhala'].tolist()))
similarity_scores_poet = util.pytorch_cos_sim(query_embeddings, model.encode(dfr_ranking['poet'].tolist()))
similarity_scores_poem = util.pytorch_cos_sim(query_embeddings, model.encode(dfr_ranking['poem'].tolist()))
similarity_scores_poem_name = util.pytorch_cos_sim(query_embeddings, model.encode(dfr_ranking['poem name'].tolist()))

terms_score_multiplier = 3
poem_score_multiplier = 1
poet_score_multiplier = 1
poem_name_score_multiplier = 1

combined_scores = np.mean([
    terms_score_multiplier * similarity_scores_terms.cpu().numpy(),
    poet_score_multiplier * similarity_scores_poet.cpu().numpy(),
    poem_score_multiplier * similarity_scores_poem.cpu().numpy(),
    poem_name_score_multiplier * similarity_scores_poem_name.cpu().numpy()
    ], axis=0)

In [ ]:
combined_scores[0][0]

1.1071393

In [ ]:
(terms_score_multiplier * similarity_scores_terms.cpu().numpy()[0][0] + poet_score_multiplier * similarity_scores_poet.cpu().numpy()[0][0] + poem_score_multiplier * similarity_scores_poem.cpu().numpy()[0][0] + poem_name_score_multiplier * similarity_scores_poem_name.cpu().numpy()[0][0]) / 4

1.107139453291893

In [ ]:
combined_scores.shape

(1, 50)

In [ ]:
dfr_ranking['Combined_Score'] = combined_scores[0]
retrieved_data = dfr_ranking.sort_values(by='Combined_Score', ascending=False)

# Ranked dfr_ranking based on combined similarity scores
retrieved_data

,_score,poem name,poet,poem number,poem,metaphorical terms sinhala,metaphorical terms english,metaphorical meaning sinhala,metaphorical meaning english,Combined_Score
17,0.914501,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සෙත් සිරි දෙන මහ ගුණ මුහුදාණන්\nසත් හට වන බව ද...,මහ ගුණ මුහුදාණන්,great ocean of noble qualities,මහා සාගරය මෙන් ඉමක් කොනක් නැති උතුම් ගුණ,Noble qualities that are as endless as the gre...,1.273188
22,0.909364,කවිසිළුමිණ,දෙවන පණ්‌ඩිත පරාක්‍රමබාහු රජු,30,සෝ පොදනඳ වුවන් විහිදි ගජනත්යතුරෙන්\nසී මැදුරෙ ...,සෝ පොදනඳ වුවන් විහිදි ගජනත්යතුරෙන්,Wet from the splashes scattered by the elephan...,ඇතුන්ගේ හොඬෙන් විසිරුණා වූ පොදින් තෙත් වූවාසේ ...,The way the woman's face was wet with drops of...,1.262462
25,0.906899,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,17,සැබැවින් බණ ඇසුවොත් මුණිඳුන් කී\nගොළුවන් බිහිර...,ගොළුවන් බිහිරන් සේ,dumb or deaf,ගොළුවන් බිහිරන් මෙන් නොවී,Don't be like the dumb and deaf,1.233857
23,0.907918,කවිසිළුමිණ,දෙවන පණ්‌ඩිත පරාක්‍රමබාහු රජු,19,සුනද මිණි වෙණෙ කලුන් රතත් තඹරට\nරැවැ වතළ බිඟු ...,රැවැ වතළ බිඟු වැල,The swarm of bees came buzzing,මැණික් වීණාවේ තන්තු කැඩී ගිය ද එහි සිරියාව දිස...,Even though the strings of the jeweled lyre we...,1.233574
19,0.911004,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,1,සෙත් සිරි දෙන මහ ගුණ මුහුදාණන්\nසත් හට වන බව ද...,දුකට වෙදාණන්,physician for sadness,ලෝක සත්ත්වයාට ඇතිවන ලෙඩ දුක් මරණය ආදී සසර දුකට...,He is like a great physician for the world's s...,1.231431
10,0.921780,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,46,බණ නම් නිබොරුය මුනිවර යන්නේ\nකය නම් නොතිරය අදහ...,පණ නම් තණ අග පිණි බිදු වැන්නේ,A man's soul is like a drop of dew on a blade ...,මිනිසාගේ ප්‍රාණය තණ කොළයක් අග ඇති පිනි බිංදුවක...,Man's soul is as fickle as a drop of dew on a ...,1.220009
40,0.893058,කවිසිළුමිණ,දෙවන පණ්‌ඩිත පරාක්‍රමබාහු රජු,7,වී කුසාවත් නම් නුවරෙක් උපත් සත්සර\nපිරිබෝ දාසි...,පිරිබෝ දාසිරි නදන් පියුමෙව් බඹ තෙලේ දුනු,A lotus that originated in the world of Brahma...,පාරිභෝගික ශ්‍රී ධාතුන්ට නිධානයක් වූ මහාසත්වයාට...,"A city called 'Kusavati', like a lotus that ar...",1.219209
4,0.932529,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,2,නිදහස මහ මුහුදක් වේ\nඑහි උල්පත පුත නුඹ වේ\nඒ බ...,එහි උල්පත,The source of the great sea,නිදහස නැමැති සාගරයේ උල්පත කුඩා දරුවායි,The source of the ocean of freedom is the litt...,1.210967
31,0.902440,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,10,සිනිඳු මොළොක් බව මලෙහී\nපිරිසුදු බව පිනි බිතෙහ...,සිනිඳු මොළොක් බව මලෙහී,Soft and tender of a flower,දරුවෙකුගේ හදවත මලක් මෙන් මෘදුයි,the heart of a child is soft and tender as a f...,1.210764
2,0.934001,ලෝවැඩ සඟරාව,වීදාගම මෙත් හිමිපාණන්,84,අවිස නොවන බස් කණට යවුල් ලෙස\nසිතස තොසින් කිය ක...,කණට යවුල් ලෙස,joke to the ears,කන්වලට යවුල් වැනි වූ රළු පරුෂ වචන,Harsh and harsh words that were like a joke to...,1.210443


# App

In [ ]:
# !pip install streamlit
# !pip install sentence_transformers
# !pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.0 MB/s eta 0:0

In [1]:
%%writefile app.py

import streamlit as st
import torch
import pandas as pd
import numpy as np
import json
import os
import uuid
import re

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers

from tqdm.auto import tqdm
tqdm.pandas()

st.set_page_config(page_title="Metaphor Mystique", layout="wide", initial_sidebar_state="expanded")

# UI text strings
page_title = "Metaphor Mystique"
page_helper = "Discover metaphors and meanings in ancient Sri Lankan poems"
empty_search_helper = "Select search terms and enter the metaphor in your own way"
category_list_header = "Search terms"
borough_search_header = "Select a borough"
term_search_header = "Select search terms"
semantic_search_header = "What metaphor are you looking for?"
semantic_search_placeholder = "මහ ගුණ මුහුදාණන්"
search_label = "Search metaphor in poems"
venue_list_header = "Venue details"

# Handler functions
def handler_load_searchterms():
    """
    Load search terms for the selected borough and update session state.
    """
    searchterms = [
        {'TERM': 'Poem Name'},
        {'TERM': 'Poet'},
        {'TERM': 'Poem'},
        {'TERM': 'Metaphorical Term'}
    ]
    st.session_state.searchterms_list = [term['TERM'] for term in searchterms]


def handler_search_metaphor():
    """
    Search for metaphor based on user query and update session state with results.
    """
    try:
        model = SentenceTransformer('Ransaka/sinhala-roberta-sentence-transformer')
        user_metaphor_query = st.session_state.user_metaphor_query
        user_metaphor_embeddings = model.encode([user_metaphor_query])
        encod_np_array = np.array(user_metaphor_embeddings)
        encod_list = encod_np_array.tolist()
        query_embeddings = encod_list[0]

        # Connect to elastic search
        try:
        #   ENDPOINT = "https://55d6-212-104-225-107.ngrok.io:443"
          ENDPOINT = "http://localhost:9200"
          USERNAME = "elastic"
          PASSWORD = "WRpT827K5LnRWFBUZh6r"

          es = Elasticsearch(hosts=[ENDPOINT],  http_auth=(USERNAME, PASSWORD), timeout=300)
          es.ping()
        except:
          st.error(f"⚠️ Elasticsearch connection failed at ENDPOINT: {ENDPOINT}.  \n  \nPlease try a different query.")

        # Build query
        knn_search_params = {
            "knn": {
                "field": "embeddings",
                "query_vector": query_embeddings,
                # @TODO: get from input
                "k": 50,
                "num_candidates": 1000
            },
            "_source": [
                "poem_name",
                "poet",
                "poem_number",
                "poem",
                "metaphorical_terms_si",
                "metaphorical_terms_en",
                "metaphorical_meaning_si",
                "metaphorical_meaning_en"
                ]
        }

        # Perform the KNN search
        query_results = es.knn_search(index="poem", body=knn_search_params)

        # Create a pandas dataframe from query results
        query_result_data = []
        for hit in query_results['hits']['hits']:
            _score = hit['_score']
            poem_name = hit['_source']["poem_name"]
            poet = hit['_source']["poet"]
            poem_number = hit['_source']["poem_number"]
            poem = hit['_source']["poem"]
            metaphorical_terms_si = hit['_source']["metaphorical_terms_si"]
            metaphorical_terms_en = hit['_source']["metaphorical_terms_en"]
            metaphorical_meaning_si = hit['_source']["metaphorical_meaning_si"]
            metaphorical_meaning_en = hit['_source']["metaphorical_meaning_en"]

            entry = {
                "_score": _score,
                "poem name": poem_name,
                "poet": poet,
                "poem number": poem_number,
                "poem": poem,
                "metaphorical terms sinhala": metaphorical_terms_si,
                "metaphorical terms english": metaphorical_terms_en,
                "metaphorical meaning sinhala": metaphorical_meaning_si,
                "metaphorical meaning english": metaphorical_meaning_en
            }

            query_result_data.append(entry)

        # Create a pandas DataFrame
        query_result_dataframe = pd.DataFrame(query_result_data)
        # query_result_dataframe.sort_values(by='_score', ascending=False)

        similarity_scores_terms = util.pytorch_cos_sim(query_embeddings, model.encode(query_result_dataframe['metaphorical terms sinhala'].tolist()))
        similarity_scores_poet = util.pytorch_cos_sim(query_embeddings, model.encode(query_result_dataframe['poet'].tolist()))
        similarity_scores_poem = util.pytorch_cos_sim(query_embeddings, model.encode(query_result_dataframe['poem'].tolist()))
        similarity_scores_poem_name = util.pytorch_cos_sim(query_embeddings, model.encode(query_result_dataframe['poem name'].tolist()))

        terms_score_multiplier = 1
        poem_score_multiplier = 1
        poet_score_multiplier = 1
        poem_name_score_multiplier = 1
        if "terms_selection" in st.session_state and len(st.session_state.terms_selection) > 0:
          if 'Poem Name' in st.session_state.terms_selection:
            poem_name_score_multiplier=3
          if 'Poet' in st.session_state.terms_selection:
            poet_score_multiplier=3
          if 'Poem' in st.session_state.terms_selection:
            poem_score_multiplier=3
          if 'Metaphorical Term' in st.session_state.terms_selection:
            terms_score_multiplier=3

        combined_scores = np.mean([
            terms_score_multiplier * similarity_scores_terms.cpu().numpy(),
            poet_score_multiplier * similarity_scores_poet.cpu().numpy(),
            poem_score_multiplier * similarity_scores_poem.cpu().numpy(),
            poem_name_score_multiplier * similarity_scores_poem_name.cpu().numpy()
            ], axis=0)

        query_result_dataframe['Combined_Score'] = combined_scores[0]
        retrieved_data = query_result_dataframe.sort_values(by='Combined_Score', ascending=False)
        st.session_state.suggested_metaphors = retrieved_data

    except Exception as e:
        st.error(f"{str(e)}")

# UI elements
def render_cta_link(url, label, font_awesome_icon):
    st.markdown('<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">', unsafe_allow_html=True)
    button_code = f'''<a href="{url}" target=_blank><i class="fa {font_awesome_icon}"></i> {label}</a>'''
    return st.markdown(button_code, unsafe_allow_html=True)

def render_search():
    """
    Render the search form in the sidebar.
    """
    search_disabled = True
    with st.sidebar:
        if "searchterms_list" in st.session_state and len(st.session_state.searchterms_list) > 0:
            st.multiselect(label=term_search_header, options=(st.session_state.searchterms_list), key="terms_selection", max_selections=3)

        st.text_input(label=semantic_search_header, placeholder=semantic_search_placeholder, key="user_metaphor_query")

        if "user_metaphor_query" in st.session_state and st.session_state.user_metaphor_query != "":
            search_disabled = False

        st.button(label=search_label, key="metaphor_search", disabled=search_disabled, on_click=handler_search_metaphor)

        st.write("---")
        render_cta_link(url="https://twitter.com/dclin", label="Let's connect", font_awesome_icon="fa-twitter")
        render_cta_link(url="https://linkedin.com/in/d2clin", label="Let's connect", font_awesome_icon="fa-linkedin")

def render_styles():
    styles = '''
        <style>
            * {
              box-sizing: border-box;
            }

            /* Custom CSS for the card component */
            .card {
                display: flex;
                border: 1px solid #333;
                box-shadow: 0 0 10px rgba(51, 51, 51, 0.2);
                padding: 0.5rem;
                border-radius: .1rem;
                background: rgba(17, 17, 17, 0.7);
            }

            .card-content {
                padding: 0 1rem;
                flex: 1;
            }

            .card-title {
                font-size: 1.5rem;
            }

            .card-subtitle {
                color: #ddd;
            }

            .card-media {
                display: flex;
                min-width: 40%;
                background-image: url('https://previews.123rf.com/images/peekeedee1/peekeedee11906/peekeedee1190600406/125248671-old-paper-texture-vintage-paper-background-or-texture-brown-paper-texture.jpg');
                background-size: cover;
                background-repeat: no-repeat;
                opacity: 0.7;
                color: #000;
                font-weight: bold;
                padding: 1rem;
                border-radius: .2rem;
                align-items: center;
                justify-content: center;
                margin: 1rem;
                border: 1px solid #aaa;
                box-shadow: 0 0 10px rgba(170, 170, 170, 0.2);

            }

            .meaning-content {
              padding-top: 2rem;
              align-items: center;
              word-wrap: break-word;
            }

            .meaning-content * {
              margin: 0;
              align-items: center;
            }

            .highlighted {
              background-color: yellow;
            }


            hr {
              box-shadow: 0 0 5px rgba(51, 51, 51, 0.2);
              margin: 0.6rem 0 !important;
            }
        </style>
    '''
    st.markdown(styles, unsafe_allow_html=True)

def render_card(poem_name, poet, poem, meaning_si, meaning_en):
    body = f'''
        <div class="card">
            <div class="card-content">
                <div class="card-title">{poem_name}</div>
                <div class="card-subtitle">{poet}</div>
                <div class="meaning-content">
                    <hr>
                    <p>{meaning_si}</p>
                    <hr>
                    <p>{meaning_en}</p>
                    <hr>
                </div>
            </div>
            <div class="card-media">
              <div>
                {poem}
              </div>
            </div>
        </div>
    '''
    st.markdown(body, unsafe_allow_html=True)
    st.write('\n')

def render_search_result():
    """
    Render the search results on the main content area.
    """
    # Number of entries per page
    entries_per_page = 5

    # Calculate the total number of pages
    total_pages = (len(st.session_state.suggested_metaphors) - 1) // entries_per_page + 1

    # Get the current page from the URL query parameter
    current_page = st.session_state.get('current_page', 1)

    # Create a paginated DataFrame
    start_idx = (current_page - 1) * entries_per_page
    end_idx = start_idx + entries_per_page
    paginated_suggested_metaphors = st.session_state.suggested_metaphors.iloc[start_idx:end_idx]

    render_styles()
    for index, row in paginated_suggested_metaphors.iterrows():
        render_card(
            row['poem name'],
            row['poet'],
            find_term_in_sentence(row['poem'].replace('\n', '<br>'), row["metaphorical terms sinhala"]),
            row['metaphorical meaning sinhala'], row['metaphorical meaning english']
        )

    # Pagination controls
    col1, col2= st.columns(2)

    with col1:
        if st.button("Previous Page", key='prev_page'):
            current_page = max(current_page - 1, 1)
    with col2:
        if st.button("Next Page", key='next_page'):
            current_page = min(current_page + 1, total_pages)

    # Store the current page in session state
    st.session_state.current_page = current_page

def find_term_in_sentence(sentence, term):
    # Replace "<br>"" with " <br>"
    sentence = re.sub(r'(<br>)', r' <br>', sentence)
    # Create a regex pattern that allows "<br>" anywhere in the term
    pattern = re.compile(r'(<br>|\s*)?'.join(map(re.escape, term)))

    # Search for the pattern in the sentence
    match = pattern.search(sentence)

    if match:
        matched_term =  match.group(0)
        return sentence.replace(matched_term, f'<span class="highlighted">{matched_term}</span>')
    else:
        return sentence

if "searchterms_list" not in st.session_state:
    handler_load_searchterms()
render_search()

st.title(page_title)
st.write(page_helper)
st.write("---")

if "suggested_metaphors" not in st.session_state:
    st.write(empty_search_helper)
else:
    render_search_result()

Overwriting app.py


In [1]:
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.10.5.112:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
/Users/nimsarafernando/Desktop/DM-Project/app.py:68: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(hosts=[ENDPOINT],  http_auth=(USERNAME, PASSWORD), timeout=300)
/Users/nimsarafernando/Desktop/DM-Project/app.py:68: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(hosts=[ENDPOINT],  http_auth=(USERNAME, PASSWORD), timeout=300)
/Users/nimsarafernando/Desktop/DM-Project/app.py:95: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  query_results = es.knn_search(index="poem", body=knn_search_params)
/Users/nimsarafernando/Desktop/DM-Project/app.py

In [ ]:
!streamlit run app.py&>/dev/null&

In [2]:
!pgrep streamlit

In [ ]:
!streamlit run app.py & npx localtunnel  --port 8501

In [ ]:
!streamlit run app.py & npx localtunnel  --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.229.67.155:8501

npx: installed 22 in 4.36s
your url is: https://busy-teams-wave.loca.lt
/content/app.py:67: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(hosts=[ENDPOINT],  http_auth=(USERNAME, PASSWORD), timeout=300)
/content/app.py:67: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(hosts=[ENDPOINT],  http_auth=(USERNAME, PASSWORD), timeout=300)
/content/app.py:94: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  query_results = es.knn_search(index="poem", body=knn_search_params)
/content/app.py:67: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(hosts=[ENDPOINT],  http_auth=(USERNAME, PASSWORD), timeout